Stock signal scanner based on the 3x Bollinger MACD Strategy. 

In [1]:
#!pip install yfinance --upgrade
#!pip install talib-binary
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
#!pip install Ta-Lib

In [2]:
import datetime as dt
from datetime import datetime, timedelta
import time
import json
import requests

import pandas as pd
import numpy as np

import yfinance as yf
import talib as ta

**1) Defining Basic Parameters & Get OHLC Data**

In [3]:
# tickers for the signal screener list
ticker_list = ['TQQQ', 'SQQQ', 'SPXL', 'SPXS', 'TZA', 'TNA', 'SOXL', 'SOXS', 'FAS', 'FAZ', 'LABD', 'LABU', 'EDZ', 'EDC']

# define all date time related metrics
# for yfinance data retrival
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=181)).strftime('%Y-%m-%d')
interval = '1d'

# for while loop operation
day_of_week = datetime.now().weekday()
now_hour = datetime.now().hour
now_minute = datetime.now().minute

**2) Sending Messages to Telegram Account**

In [4]:
def send_to_telegram(message):

    apiToken = '5xxxxxxxxx:AAFxxxx_cnM0qQNWDpa6y0rPZGyqfmQ'
    chatID = '-7xxxxxxx'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)

**3) Executing the Screener within Specific Time Parameters**

In [ ]:
while True: 
  # start screening everyday, 5 minutes before market opens
  if (day_of_week < 6) and (now_hour == 2) and (now_minute > 24):

    for ticker in ticker_list:
      # rename dataframe to ticker, download data from yfinance
      globals()[ticker] = pd.DataFrame
      globals()[ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
      
      # add MACD and Bollinger indicator from TA-Lib
      globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
      globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) 

      # data cleaning - drop empty lines, reset index
      globals()[ticker] = globals()[ticker].dropna()
      globals()[ticker] = globals()[ticker].reset_index()    
      globals()[ticker]['Date'] = globals()[ticker]['Date'].dt.strftime('%Y-%m-%d')

      # create signal indicators
      last_index      = int(globals()[ticker].last_valid_index())
      today           = end_date
      now_Date        = globals()[ticker].at[last_index,'Date']
      now_MACD        = globals()[ticker].at[last_index,'MACD']
      now_MACD_signal = globals()[ticker].at[last_index,'MACD_signal']
      now_MACD_hist   = globals()[ticker].at[last_index,'MACD_hist']
      now_BBand_L     = globals()[ticker].at[last_index,'BBand_L']
      prev_close      = globals()[ticker].at[last_index,'Close']
      now_open        = globals()[ticker].at[last_index,'Open']

      # scan and send signal to telegram
      if now_Date != today:
        if ((now_MACD > now_MACD_signal) and (now_MACD_hist >0)):
          message = now_Date + ' - ' + ticker + ': target buy <$' + str(round(now_BBand_L,2)) + ', prev close $' + str(round(prev_close,2))
          send_to_telegram(message)
      elif now_Date == today:
        if ((now_MACD > now_MACD_signal) and (now_MACD_hist >0) and (now_open < now_BBand_L)):
          message = now_Date + ' - ' + ticker + ': target buy <$' + str(round(now_BBand_L,2)) + ', open $' + str(round(now_open,2))
          send_to_telegram(message) 

    time.sleep(300)
  
  else:
    time.sleep(1800) 